In [0]:
from __future__ import print_function

In [0]:
max_features = 15000

# cut texts after this number of words (among top max_features most common words)
maxlen = 100
batch_size = 32

In [3]:
from keras.datasets import imdb

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Using TensorFlow backend.


Loading data...
17465344/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [4]:
from keras.preprocessing import sequence

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen,padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen,padding='post')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Reshape, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten
from keras.regularizers import l2,l1_l2


In [18]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
'''
#model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.55,
    kernel_regularizer=l1_l2(l1=1e-4, l2=1e-3),
    recurrent_regularizer=l2(5e-3),
    bias_regularizer=l2(1e-3),
    activity_regularizer=l2(1e-4),
    return_sequences=True))
#model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.4,
    kernel_regularizer=l1_l2(l1=1e-3, l2=5e-3),
    recurrent_regularizer=l2(1e-2),
    bias_regularizer=l2(5e-3),
    activity_regularizer=l2(1e-3)))
#model.add(Reshape((64,1)))
model.add(Conv1D(128,5,padding="valid",activation="relu"))
model.add(Conv1D(128,5,padding="valid",activation="relu"))
model.add(MaxPooling1D(2))
model.add(Conv1D(256,5,padding="valid",activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(128,activation='linear'))
model.add(Dense(1, activation='sigmoid'))

Build model...


In [0]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [20]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         1920000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 256)         164096    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)              

In [21]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))


Train...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 136s 5ms/step - loss: 0.4961 - accuracy: 0.7313 - val_loss: 0.3692 - val_accuracy: 0.8333
Epoch 2/15
25000/25000 [==============================] - 136s 5ms/step - loss: 0.2628 - accuracy: 0.8945 - val_loss: 0.3676 - val_accuracy: 0.8411
Epoch 3/15
25000/25000 [==============================] - 136s 5ms/step - loss: 0.1317 - accuracy: 0.9511 - val_loss: 0.4557 - val_accuracy: 0.8338
Epoch 4/15
25000/25000 [==============================] - 136s 5ms/step - loss: 0.0597 - accuracy: 0.9796 - val_loss: 0.6744 - val_accuracy: 0.8152
Epoch 5/15
25000/25000 [==============================] - 137s 5ms/step - loss: 0.0345 - accuracy: 0.9878 - val_loss: 0.8412 - val_accuracy: 0.8210
Epoch 6/15
25000/25000 [==============================] - 136s 5ms/step - loss: 0.0248 - accuracy: 0.9911 - val_loss: 1.1508 - val_accuracy: 0.7918
Epoch 7/15
25000/25000 [==============================] - 136s

In [0]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [0]:
model.save('20-05-18-1000.h5')